<a href="https://colab.research.google.com/github/oliveiraprg/text-summarization-with-python/blob/main/sumarizacao_com_similaridade_do_cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sumarização com similaridade do cosseno

## Preparação do ambiente

In [9]:
import re
import nltk
import string
from nltk.cluster.util import cosine_distance
import networkx as nx
import numpy as np

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [4]:
def pre_processamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)
  tokens = [palavra for palavra in tokens if palavra not in stop_words and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

In [5]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [6]:
texto_original = re.sub(r'\s+', ' ', texto_original)

## Função para calcular similaridade entre sentenças

In [7]:
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto_original)]
sentencas_formatadas = [pre_processamento(sentencas_original) for sentencas_original in sentencas_originais]

In [10]:
def calcula_similaridade_sentencas(primeira_sentenca, segunda_sentenca):
  primeira_palavras = [palavra for palavra in nltk.word_tokenize(primeira_sentenca)]
  segunda_palavras = [palavra for palavra in nltk.word_tokenize(segunda_sentenca)]
  todas_palavras = list(set(primeira_palavras + segunda_palavras))
  primeiro_vetor = [0] * len(todas_palavras)
  segundo_vetor = [0] * len(todas_palavras)

  for palavra in primeira_palavras:
    primeiro_vetor[todas_palavras.index(palavra)] += 1 

  for palavra in segunda_palavras:
    segundo_vetor[todas_palavras.index(palavra)] += 1 

  return 1 - cosine_distance(primeiro_vetor, segundo_vetor)

##Função para gerar a matriz de similaridade

In [11]:
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
  for linha in range(len(sentencas)):
    for coluna in range(len(sentencas)):
      if linha == coluna:continue
      matriz_similaridade[linha][coluna] = calcula_similaridade_sentencas(sentencas[linha], sentencas[coluna])
  return matriz_similaridade

In [12]:
calcula_matriz_similaridade(sentencas_formatadas)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])